# QIR generation with Q# #

_Tutorials are narrated examples of using QIR as a part of some workflow or with a particular set of tools._

- using the `%qir` magic command in notebooks: https://docs.microsoft.com/en-us/qsharp/api/iqsharp-magic/qir
- use the project file flag to specify QIR compilation of a Q# program: https://github.com/microsoft/qsharp-compiler/tree/main/src/QsCompiler/QirGeneration

In this section, QIR is generated from Q# code for teleportation to transfer the binary representation of “hello world!”, followed with ASCII encoding. Teleportation technique contains heterogeneous instructions at the compilation time. The interdependent conditionals following the measurement of the source and the auxiliary qubits in teleportation determine the next quantum gate operations on the target qubit.

Computations involving heterogeneous instructions at compile time can benefit from QIR enabled compilers. QIR is generated in LLVM, which is backed by decades of classical compilation research and development. Hybrid quantum algorithms were [defined](https://arxiv.org/pdf/2207.06850.pdf) to rely on both classical and quantum resources to fulfill the abstract computational model of the algorithm. In the next series of tutorials, a hybrid classical-quantum algorithm such as VQE or HHL (?) will be used to demonstrate further capabilities of QIR. 


In [6]:
// Building on https://github.com/microsoft/Quantum/blob/main/samples/getting-started/teleportation/TeleportationSample.qs
// Copyright (c) Microsoft Corporation.
// Licensed under the MIT License.
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Core;
open Microsoft.Quantum.Arrays;

In [7]:
// Converts data type: int to bool 
operation int2bool(bit: Int) : Bool {
        if (bit == 1){return true;}
        else {return false;}
    }
// Converts data type: bool to int 
 operation bool2int(msg: Bool) : Int {
        if (msg == true){return 1;}
        else {return 0;}
    }
// Pure quantum section of teleportation technique 
operation Teleport (msg : Qubit, target : Qubit) : Unit {
    use register = Qubit();

    H(register);
    CNOT(register, target);

    CNOT(msg, register);
    H(msg);


    if (MResetZ(msg) == One) { Z(target); }

    if (IsResultOne(MResetZ(register))) { X(target); }
}
// Interdependent conditional section of teleportation technique 
operation TeleportClassicalMessage (message : Bool) : Bool {
   
    use (msg, target) = (Qubit(), Qubit());

   
    if (message) {
        X(msg);
    }

 
    Teleport(msg, target);

    
    return MResetZ(target) == One;
}

int2bool, bool2int, Teleport, TeleportClassicalMessage

In [8]:
// Runs teleporation with 2 loops: classical and hetrogenous 
operation Greetings() : Unit { 
    mutable binary=[1, 0, 1, 0, 0];
    let msg = ForEach(int2bool, binary);
    let transferred= ForEach(TeleportClassicalMessage, msg);
    for index in 0 .. Length(transferred)-1{
        Message($"{transferred[index]}");
    }     
}

Greetings

In [9]:
%simulate Greetings

True
False
True
False
False


()

In [10]:
%qir Greetings

; ModuleID = '/tmp/tmpcyZsbq.bc'

%Tuple = type opaque
%Array = type opaque
%Callable = type opaque
%String = type opaque
%Qubit = type opaque
%Result = type opaque

@SNIPPET__int2bool__FunctionTable = internal constant [4 x void (%Tuple*, %Tuple*, %Tuple*)*] [void (%Tuple*, %Tuple*, %Tuple*)* @SNIPPET__int2bool__body__wrapper, void (%Tuple*, %Tuple*, %Tuple*)* null, void (%Tuple*, %Tuple*, %Tuple*)* null, void (%Tuple*, %Tuple*, %Tuple*)* null]
@SNIPPET__TeleportClassicalMessage__FunctionTable = internal constant [4 x void (%Tuple*, %Tuple*, %Tuple*)*] [void (%Tuple*, %Tuple*, %Tuple*)* @SNIPPET__TeleportClassicalMessage__body__wrapper, void (%Tuple*, %Tuple*, %Tuple*)* null, void (%Tuple*, %Tuple*, %Tuple*)* null, void (%Tuple*, %Tuple*, %Tuple*)* null]
@0 = internal constant [5 x i8] c"true\00"
@1 = internal constant [6 x i8] c"false\00"
@2 = internal constant [3 x i8] c"()\00"

define internal void @ENTRYPOINT__Greetings__body() {
entry:
  call void @SNIPPET__Greetings__body()
  ret void
}

define internal void @SNIPPET__Greetings__body() {
entry:
  %binary = alloca %Array*, align 8
  %0 = call %Array* @__quantum__rt__array_create_1d(i32 8, i64 5)
  %1 = call i8* @__quantum__rt__array_get_element_ptr_1d(%Array* %0, i64 0)
  %2 = bitcast i8* %1 to i64*
  %3 = call i8* @__quantum__rt__array_get_element_ptr_1d(%Array* %0, i64 1)
  %4 = bitcast i8* %3 to i64*
  %5 = call i8* @__quantum__rt__array_get_element_ptr_1d(%Array* %0, i64 2)
  %6 = bitcast i8* %5 to i64*
  %7 = call i8* @__quantum__rt__array_get_element_ptr_1d(%Array* %0, i64 3)
  %8 = bitcast i8* %7 to i64*
  %9 = call i8* @__quantum__rt__array_get_element_ptr_1d(%Array* %0, i64 4)
  %10 = bitcast i8* %9 to i64*
  store i64 1, i64* %2, align 4
  store i64 0, i64* %4, align 4
  store i64 1, i64* %6, align 4
  store i64 0, i64* %8, align 4
  store i64 0, i64* %10, align 4
  store %Array* %0, %Array** %binary, align 8
  call void @__quantum__rt__array_update_alias_count(%Array* %0, i32 1)
  %11 = call %Callable* @__quantum__rt__callable_create([4 x void (%Tuple*, %Tuple*, %Tuple*)*]* @SNIPPET__int2bool__FunctionTable, [2 x void (%Tuple*, i32)*]* null, %Tuple* null)
  %msg = call %Array* @Microsoft__Quantum__Arrays___2f0d187e7e834572997fb8555ba04655_ForEach__body(%Callable* %11, %Array* %0)
  call void @__quantum__rt__array_update_alias_count(%Array* %msg, i32 1)
  %12 = call %Callable* @__quantum__rt__callable_create([4 x void (%Tuple*, %Tuple*, %Tuple*)*]* @SNIPPET__TeleportClassicalMessage__FunctionTable, [2 x void (%Tuple*, i32)*]* null, %Tuple* null)
  %transferred = call %Array* @Microsoft__Quantum__Arrays___8838e09e7d6a472184c4e9384893b4a9_ForEach__body(%Callable* %12, %Array* %msg)
  call void @__quantum__rt__array_update_alias_count(%Array* %transferred, i32 1)
  %13 = call i64 @__quantum__rt__array_get_size_1d(%Array* %transferred)
  %14 = sub i64 %13, 1
  br label %header__1

header__1:                                        ; preds = %exiting__1, %entry
  %index = phi i64 [ 0, %entry ], [ %22, %exiting__1 ]
  %15 = icmp sle i64 %index, %14
  br i1 %15, label %body__1, label %exit__1

body__1:                                          ; preds = %header__1
  %16 = call i8* @__quantum__rt__array_get_element_ptr_1d(%Array* %transferred, i64 %index)
  %17 = bitcast i8* %16 to i1*
  %18 = load i1, i1* %17, align 1
  br i1 %18, label %condTrue__1, label %condFalse__1

condTrue__1:                                      ; preds = %body__1
  %19 = call %String* @__quantum__rt__string_create(i8* getelementptr inbounds ([5 x i8], [5 x i8]* @0, i32 0, i32 0))
  br label %condContinue__1

condFalse__1:                                     ; preds = %body__1
  %20 = call %String* @__quantum__rt__string_create(i8* getelementptr inbounds ([6 x i8], [6 x i8]* @1, i32 0, i32 0))
  br label %condContinue__1

condContinue__1:                                  ; preds = %condFalse__1, %condTrue__1
  %21 = phi %String* [ %19, %condTrue__1 ], [ %20, %condFalse__1 ]
  call void @__quantu